<a href="https://colab.research.google.com/github/lanej5/mlb/blob/main/postseason_race.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

done:
- plotting teams by wins over 500


todo plotting:
- buttons to filter by division
- hover text for games
- dots for division leaders
- dots for wildcard leaders
- names for teams

todo data:
- store in the cloud (how?)

todo dashboard:
- testing
- deploying
- monitoring

future ideas:
- projections with error bars
- 

Resources:
- [cool postseason visualiztion with tableau](https://www.ryansleeper.com/2017-mlb-standings-reinvented/)
- [pybaseball](https://github.com/jldbc/pybaseball)
- [Fan graphs](https://www.fangraphs.com/)

In [1]:
! pip install pybaseball

     |████████████████████████████████| 415 kB 8.1 MB/s 
     |████████████████████████████████| 291 kB 41.4 MB/s 
     |████████████████████████████████| 856 kB 47.0 MB/s 


In [3]:
from pybaseball import schedule_and_record
import time

al = {
    'al_east':['NYY', 'TBR', 'TOR', 'BAL', 'BOS'],
    'al_central':['CHW', 'CLE', 'DET', 'KCR', 'MIN'],
    'al_west':['LAA', 'OAK', 'SEA', 'TEX', 'HOU']
}

def scrape_records(year):
    records = {}

    for div, teams in al.items():
        for team in teams:
            records[team] = schedule_and_record(year, team)
            time.sleep(2)

    return records


In [4]:
al_records = scrape_records(2022)

/usr/local/lib/python3.7/dist-packages/pybaseball/team_results.py:70: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('',1) #not a useful column
/usr/local/lib/python3.7/dist-packages/pybaseball/team_results.py:83: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('Streak2',1)


In [ ]:
data.Date = data.Date.astype(str)

In [76]:
import pandas as pd
from datetime import datetime

months = {'Jan': 1, 'Feb':2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

def process_records(records):
    processed_records = {}
    for team, record in records.items():
        processed_record = record.loc[~record.R.isna(), ['Date', 'W/L']].copy()
        processed_record.Date = processed_record.Date.str.split(' ') \
          .apply(lambda x: pd.Timestamp(datetime(2022, months[x[1]], int(x[2]))))
        processed_record['W/L'] = processed_record['W/L'].apply(lambda x: 1 if x == 'W' else -1)
        processed_record = processed_record.groupby('Date', as_index=False)['W/L'].sum()
        processed_record.rename(columns={'W/L': team}, inplace=True)
        processed_records[team] = processed_record

    return processed_records

In [78]:
processed_records = process_records(al_records)

In [ ]:
processed_records['TOR']

In [109]:
season = pd.date_range(start=datetime(2022, 4, 7), end=datetime(2022, 10, 2))
played_season = pd.to_datetime(pd.date_range(start=datetime(2022, 4, 7), end=pd.Timestamp.today()))

def merge_records(processed_records):
    played_season = pd.date_range(start=datetime(2022, 4, 7), end=pd.Timestamp.today())
    df = pd.DataFrame(played_season.astype('datetime64[ns]'), columns=['Date'])
    
    for team, record in processed_records.items():
        df = pd.merge(
            df,
            record,
            how='outer',
            on='Date'
        )
    return df.set_index('Date').cumsum().ffill().fillna(0)


In [110]:
df = merge_records(processed_records)

In [111]:
df

,NYY,TBR,TOR,BAL,BOS,CHW,CLE,DET,KCR,MIN,LAA,OAK,SEA,TEX,HOU
Date,,,,,,,,,,,,,,,
2022-04-07,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,1.0
2022-04-08,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-2.0,-1.0,1.0,-1.0,2.0
2022-04-09,0.0,2.0,2.0,-2.0,-2.0,0.0,-2.0,-2.0,0.0,-2.0,-1.0,-2.0,2.0,-2.0,1.0
2022-04-10,-1.0,3.0,1.0,-3.0,-1.0,1.0,-1.0,-3.0,-1.0,-1.0,-2.0,-1.0,1.0,-1.0,2.0
2022-04-11,-2.0,2.0,2.0,-2.0,-2.0,1.0,0.0,-2.0,-2.0,0.0,-1.0,0.0,0.0,-2.0,2.0
2022-04-12,-1.0,1.0,1.0,-3.0,-1.0,2.0,1.0,-3.0,-3.0,-1.0,-2.0,-1.0,-1.0,-3.0,3.0
2022-04-13,-2.0,0.0,2.0,-4.0,0.0,3.0,2.0,-4.0,-3.0,-2.0,-2.0,0.0,-2.0,-3.0,2.0
2022-04-14,-1.0,-1.0,1.0,-4.0,0.0,2.0,2.0,-3.0,-4.0,-2.0,-3.0,1.0,-1.0,-2.0,2.0
2022-04-15,-2.0,-2.0,2.0,-5.0,-1.0,3.0,1.0,-2.0,-5.0,-1.0,-2.0,0.0,0.0,-3.0,1.0


In [112]:
import plotly.express as px

fig = px.line(df, x=df.index, y="TOR", title='Toronto Games over .500')
fig.show()

In [113]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

division_name = {
    'al_east': 'AL East',
    'al_central': 'AL Central',
    'al_west': 'AL West'
}

traces = []
team_indexes = {}
div_leader_index = {}

for i, team in enumerate(df.columns):
    
    team_indexes[team] = i
    
    traces.append(
        go.Scatter(
            x=df.index,
            y=df[team],
            visible=True,
            name=team
        )
    )

# buttons = []
# for div in results.columns.levels[0]:
#     buttons.append(
#         dict(
#             method='restyle',
#             label=division_name[div],
#             visible=True,
#             args=[{'visible':True}, div_indexes[div]],
#             args2=[{'visible':'legendonly'}, div_indexes[div]]
#         )
#     )

# create the layout 
layout = go.Layout(
    # updatemenus=[
    #     dict(
    #         type='buttons',
    #         direction='right',
    #         x=1,
    #         y=1.3,
    #         showactive=True,
    #         buttons=buttons
    #     )
    # ],
    title=dict(text='American League Postseason Race',x=0.5),
    showlegend=True
)

fig = go.Figure(data=traces,layout=layout)

fig.update_xaxes(range=[season.min(), season.max()])
fig.update_yaxes(range=[df.min().min() - 5, df.max().max() + 5])

fig.show()

In [ ]:
al = {
    'al_east':['NYY', 'TBR', 'TOR', 'BAL', 'BOS'],
    'al_central':['CHW', 'CLE', 'DET', 'KCR', 'MIN'],
    'al_west':['LAA', 'OAK', 'SEA', 'TEX', 'HOU']
}

data = [(div, team) for div in al.keys() for team in al[div] ]

american_league = pd.DataFrame(
    data,
    columns=["division", "team"]
)

american_league.head()

,division,team
0,al_east,NYY
1,al_east,TBR
2,al_east,TOR
3,al_east,BAL
4,al_east,BOS


In [ ]:
from datetime import datetime

season = pd.date_range(start=datetime(2022, 4, 7), end=datetime(2022, 10, 2))
played_season = pd.date_range(start=datetime(2022, 4, 7), end=pd.Timestamp.today())

len(season)

wins = pd.DataFrame(
    np.random.randint(low=0, high=2, size=(len(played_season), 15)),
    columns=pd.MultiIndex.from_frame(american_league),
    index=played_season
).cumsum()

games = pd.DataFrame(
    np.ones((len(played_season), 15)),
    columns=pd.MultiIndex.from_frame(american_league),
    index=played_season
).cumsum()

losses = games - wins
results = wins - losses
results.tail()

division   al_east                     al_central                     al_west  \
team           NYY  TBR  TOR  BAL  BOS        CHW  CLE  DET  KCR  MIN     LAA   
2022-05-05    -1.0  5.0 -7.0  3.0 -7.0        5.0 -3.0 -1.0 -1.0  3.0     9.0   
2022-05-06     0.0  6.0 -8.0  4.0 -6.0        4.0 -2.0 -2.0 -2.0  2.0     8.0   
2022-05-07    -1.0  7.0 -7.0  5.0 -7.0        3.0 -3.0 -3.0 -3.0  3.0     9.0   
2022-05-08     0.0  8.0 -6.0  6.0 -6.0        2.0 -4.0 -2.0 -2.0  2.0    10.0   
2022-05-09    -1.0  9.0 -5.0  7.0 -7.0        1.0 -5.0 -1.0 -1.0  3.0    11.0   

division                         
team        OAK   SEA  TEX  HOU  
2022-05-05 -7.0 -13.0 -5.0 -1.0  
2022-05-06 -8.0 -12.0 -6.0 -2.0  
2022-05-07 -7.0 -11.0 -5.0 -1.0  
2022-05-08 -6.0 -10.0 -4.0  0.0  
2022-05-09 -7.0  -9.0 -5.0  1.0

In [ ]:
for div in results.columns.levels[0]:
    print(div)

al_central
al_east
al_west


In [ ]:
results.xs('al_east', level=0, axis=1)

team,NYY,TBR,TOR,BAL,BOS
2022-04-07,1.0,-1.0,1.0,1.0,-1.0
2022-04-08,0.0,0.0,0.0,2.0,-2.0
2022-04-09,1.0,1.0,-1.0,1.0,-1.0
2022-04-10,2.0,2.0,-2.0,0.0,0.0
2022-04-11,3.0,1.0,-1.0,1.0,-1.0
...,...,...,...,...,...
2022-09-28,3.0,-3.0,-13.0,-7.0,-15.0
2022-09-29,4.0,-4.0,-14.0,-6.0,-16.0
2022-09-30,3.0,-3.0,-13.0,-5.0,-17.0
2022-10-01,2.0,-4.0,-14.0,-4.0,-18.0


In [ ]:
results.loc[:,('al_east', "TOR")]

2022-04-07     1.0
2022-04-08     0.0
2022-04-09    -1.0
2022-04-10    -2.0
2022-04-11    -1.0
              ... 
2022-09-28   -13.0
2022-09-29   -14.0
2022-09-30   -13.0
2022-10-01   -14.0
2022-10-02   -15.0
Freq: D, Name: (al_east, TOR), Length: 179, dtype: float64

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import datetime
from collections import defaultdict

division_name = {
    'al_east': 'AL East',
    'al_central': 'AL Central',
    'al_west': 'AL West'
}

traces = []
div_indexes = defaultdict(list)
div_leader_index = {}

for i, (div, team) in enumerate(results.columns):
    
    div_indexes[div].append(i)
    
    traces.append(
        go.Scatter(
            x=results.index,
            y=results.loc[:, (div, team)],
            visible=True,
            name=team
        )
    )

buttons = []
for div in results.columns.levels[0]:
    buttons.append(
        dict(
            method='restyle',
            label=division_name[div],
            visible=True,
            args=[{'visible':True}, div_indexes[div]],
            args2=[{'visible':'legendonly'}, div_indexes[div]]
        )
    )

# create the layout 
layout = go.Layout(
    updatemenus=[
        dict(
            type='buttons',
            direction='right',
            x=1,
            y=1.3,
            showactive=True,
            buttons=buttons
        )
    ],
    title=dict(text='American League Postseason Race',x=0.5),
    showlegend=True
)

fig = go.Figure(data=traces,layout=layout)

fig.update_xaxes(range=[season.min(), season.max()])
fig.update_yaxes(range=[results.min().min() - 10, results.max().max() + 10])

fig.show()